In [2]:
#for Preview version in BAM

import dotenv
import os

from genai.credentials import Credentials
from genai.schemas import GenerateParams
from genai.model import Model

dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=200,
    min_new_tokens=1,
    stop_sequences=['"""']
)

model = Model(model="meta-llama/llama-2-70b-chat",credentials=creds, params=params)
# model = Model(model="codellama/codellama-34b-instruct",credentials=creds, params=params)

In [3]:
from docx import Document

xmlstring = """
<xml>
    <interface name="source">Mulesoft API#1</interface>
    <interface name="target">Mulesoft API#2</interface>
    <function name="abc" description="def">
</xml>  
"""

prompt_template = """[INST]locate the information from the xml provide.\n
extract the information of {placeholder}.
show the extract value only.<<SYS>>
    xml:```{xml}```
    <</SYS>>
    [/INST]
    extract value:"""

# input_file_path = "/Users/yingkitw/Desktop/Incubation program deck tw/8 RAG with watsonx.pptx"
input_file_path =  "tech spec template.docx"

adoc = Document(input_file_path)

for paragraph in adoc.paragraphs:
    placeholdertext = paragraph.text
    print(placeholdertext)
    response = model.generate([prompt_template.format(placeholder=placeholdertext, xml=xmlstring[0:4000])])
    paragraph.text = response[0].generated_text
    print(paragraph.text+'->'+response[0].generated_text)

for table in adoc.tables:
    for row in table.rows:
        for cell in row.cells:
            placeholdertext = cell.text
            print(placeholdertext)
            if len(placeholdertext) > 0 and placeholdertext[0] == '[' and placeholdertext[-1] == ']':
                prompttemplate = prompt_template.format(placeholder=placeholdertext,xml=xmlstring[0:4000])
                response = model.generate([prompttemplate])
                cell.text = response[0].generated_text

output_file_path = input_file_path.replace(
        '.docx', '-out.docx')

adoc.save(output_file_path)

Tech spec template


* interface name="source": Mulesoft API#1
* interface name="target": Mulesoft API#2
* function name="abc": def->

* interface name="source": Mulesoft API#1
* interface name="target": Mulesoft API#2
* function name="abc": def



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"
[source interface]
 Mulesoft API#1-> Mulesoft API#1
[target interface]
 Mulesoft API#2-> Mulesoft API#2



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function descripti